# 🤖 RinominatorAI - Document Processing System

Sistema AI per rinominare automaticamente PDF estraendo:
- **Denominazione** (alto a sinistra)
- **Numero Documento** (centro)
- **Data Documento** (centro a destra)

---

## 📋 Istruzioni Rapide:
1. ▶️ Esegui tutte le celle in ordine
2. 📂 Carica i tuoi PDF quando richiesto
3. ⏳ Attendi l'elaborazione
4. 💾 Scarica i file rinominati


In [ ]:
# ============================================
# 1️⃣ SETUP E INSTALLAZIONE DIPENDENZE
# ============================================

print("🔧 Installazione dipendenze...")
print("⏱️ Questo richiederà 2-3 minuti...\n")

!pip install -q PyMuPDF Pillow easyocr pandas python-dateutil
!pip install -q transformers torch torchvision

print("✅ Installazione completata!\n")

In [ ]:
# ============================================
# 2️⃣ IMPORT LIBRERIE
# ============================================

import os
import re
import fitz  # PyMuPDF
from PIL import Image
import easyocr
import numpy as np
from pathlib import Path
from datetime import datetime
from dateutil import parser as date_parser
import zipfile
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerie importate con successo!")

In [ ]:
# ============================================
# 3️⃣ INIZIALIZZAZIONE OCR
# ============================================

print("🚀 Inizializzazione EasyOCR con supporto Italiano...")
print("⏱️ Primo avvio: download modelli (~100MB)...\n")

# Inizializza OCR per Italiano e Inglese con GPU
reader = easyocr.Reader(['it', 'en'], gpu=True)

print("✅ OCR pronto!\n")

In [ ]:
# ============================================
# 4️⃣ FUNZIONI PRINCIPALI
# ============================================

def pdf_to_image(pdf_path, output_folder="temp_images"):
    """Converte la prima pagina del PDF in immagine."""
    os.makedirs(output_folder, exist_ok=True)
    
    doc = fitz.open(pdf_path)
    page = doc[0]  # Prima pagina
    
    # Renderizza ad alta risoluzione per OCR migliore
    mat = fitz.Matrix(2.0, 2.0)  # 2x zoom = 144 DPI
    pix = page.get_pixmap(matrix=mat)
    
    image_path = os.path.join(output_folder, "temp_page.png")
    pix.save(image_path)
    doc.close()
    
    return image_path


def extract_text_with_positions(image_path):
    """Estrae testo con coordinate usando EasyOCR."""
    result = reader.readtext(image_path)
    
    # Formatta risultati: (bbox, text, confidence)
    extracted = []
    for detection in result:
        bbox, text, confidence = detection
        
        # Calcola centro del bounding box
        x_coords = [point[0] for point in bbox]
        y_coords = [point[1] for point in bbox]
        
        center_x = sum(x_coords) / len(x_coords)
        center_y = sum(y_coords) / len(y_coords)
        
        extracted.append({
            'text': text,
            'confidence': confidence,
            'x': center_x,
            'y': center_y,
            'bbox': bbox
        })
    
    return extracted


def detect_field_by_position(extracted_data, position_type, keywords=None):
    """Rileva campo basandosi sulla posizione e parole chiave."""
    if not extracted_data:
        return None
    
    # Calcola dimensioni documento
    max_x = max([item['x'] for item in extracted_data])
    max_y = max([item['y'] for item in extracted_data])
    
    # Definisci zone del documento
    zones = {
        'top-left': lambda x, y: x < max_x * 0.4 and y < max_y * 0.3,
        'center': lambda x, y: max_x * 0.3 < x < max_x * 0.7 and y < max_y * 0.5,
        'center-right': lambda x, y: x > max_x * 0.5 and y < max_y * 0.5
    }
    
    zone_filter = zones.get(position_type, lambda x, y: True)
    
    # Filtra per zona
    candidates = [
        item for item in extracted_data 
        if zone_filter(item['x'], item['y']) and item['confidence'] > 0.3
    ]
    
    if not candidates:
        return None
    
    # Se ci sono keywords, cerca quelle
    if keywords:
        for keyword in keywords:
            for item in candidates:
                if keyword.lower() in item['text'].lower():
                    # Cerca il valore dopo la keyword
                    idx = candidates.index(item)
                    if idx + 1 < len(candidates):
                        return candidates[idx + 1]['text']
                    return item['text']
    
    # Ritorna il testo più lungo nella zona
    return max(candidates, key=lambda x: len(x['text']))['text']


def extract_denominazione(extracted_data):
    """Estrae la denominazione del fornitore (alto a sinistra)."""
    keywords = ['denominazione', 'ragione sociale', 'ditta', 'fornitore']
    result = detect_field_by_position(extracted_data, 'top-left', keywords)
    
    if result:
        # Pulisci il testo
        result = re.sub(r'[^\w\s-]', '', result)
        return result.strip()
    
    return "Fornitore_Sconosciuto"


def extract_numero_documento(extracted_data):
    """Estrae il numero del documento (centro)."""
    keywords = ['n.', 'numero', 'n°', 'nr', 'fattura', 'doc']
    
    # Cerca pattern numerici nel centro
    center_items = [
        item for item in extracted_data
        if 'center' in str(detect_field_by_position([item], 'center'))
    ]
    
    for item in extracted_data:
        text = item['text']
        # Cerca pattern: "N. 12345" o "Fattura 12345"
        for keyword in keywords:
            if keyword in text.lower():
                # Estrai numero dopo keyword
                match = re.search(rf'{keyword}\s*[:.]?\s*(\d+[\w/-]*)', text, re.IGNORECASE)
                if match:
                    return match.group(1)
    
    # Cerca qualsiasi numero nel centro
    for item in extracted_data:
        match = re.search(r'\b\d{3,}\b', item['text'])
        if match:
            return match.group(0)
    
    return "NUM_NF"


def extract_data_documento(extracted_data):
    """Estrae la data del documento (centro-destra)."""
    keywords = ['data', 'del', 'emissione', 'date']
    
    # Pattern comuni per date italiane
    date_patterns = [
        r'\b(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})\b',  # 12/03/2024 o 12-03-24
        r'\b(\d{1,2}\s+[a-zA-Z]+\s+\d{4})\b',     # 12 marzo 2024
    ]
    
    # Cerca date nel testo
    for item in extracted_data:
        text = item['text']
        
        for pattern in date_patterns:
            match = re.search(pattern, text)
            if match:
                date_str = match.group(1)
                try:
                    # Prova a parsare la data
                    parsed_date = date_parser.parse(date_str, dayfirst=True)
                    return parsed_date.strftime('%d-%m-%Y')
                except:
                    continue
    
    return "DATA_NF"


def clean_filename(text):
    """Pulisce il testo per nome file valido."""
    # Rimuovi caratteri non validi per Windows/Linux
    text = re.sub(r'[<>:"/\\|?*]', '', text)
    # Limita lunghezza
    text = text[:100]
    return text.strip()


def process_pdf(pdf_path, output_folder):
    """Processa un singolo PDF e lo rinomina."""
    try:
        print(f"\n📄 Elaborazione: {os.path.basename(pdf_path)}")
        
        # 1. Converti prima pagina in immagine
        print("  ├─ Conversione PDF → Immagine...")
        image_path = pdf_to_image(pdf_path)
        
        # 2. OCR con rilevamento posizioni
        print("  ├─ OCR ed estrazione testo...")
        extracted_data = extract_text_with_positions(image_path)
        
        # 3. Estrai campi
        print("  ├─ Rilevamento campi...")
        denominazione = extract_denominazione(extracted_data)
        numero = extract_numero_documento(extracted_data)
        data = extract_data_documento(extracted_data)
        
        print(f"  │  ├─ Denominazione: {denominazione}")
        print(f"  │  ├─ Numero: {numero}")
        print(f"  │  └─ Data: {data}")
        
        # 4. Crea nuovo nome file
        new_filename = f"{denominazione} {numero} del {data}.pdf"
        new_filename = clean_filename(new_filename)
        
        # 5. Copia e rinomina
        output_path = os.path.join(output_folder, new_filename)
        
        import shutil
        shutil.copy2(pdf_path, output_path)
        
        print(f"  └─ ✅ Salvato come: {new_filename}")
        
        return {
            'original': os.path.basename(pdf_path),
            'new_name': new_filename,
            'denominazione': denominazione,
            'numero': numero,
            'data': data,
            'status': 'success'
        }
        
    except Exception as e:
        print(f"  └─ ❌ Errore: {str(e)}")
        return {
            'original': os.path.basename(pdf_path),
            'status': 'error',
            'error': str(e)
        }


print("✅ Funzioni caricate con successo!")

In [ ]:
# ============================================
# 5️⃣ CARICA I TUOI PDF
# ============================================

print("📂 Carica i tuoi file PDF")
print("   Puoi selezionare più file contemporaneamente\n")

uploaded = files.upload()

# Crea cartelle
input_folder = "input_pdfs"
output_folder = "output_renamed"
os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Salva file caricati
pdf_files = []
for filename in uploaded.keys():
    if filename.lower().endswith('.pdf'):
        filepath = os.path.join(input_folder, filename)
        with open(filepath, 'wb') as f:
            f.write(uploaded[filename])
        pdf_files.append(filepath)

print(f"\n✅ {len(pdf_files)} PDF caricati con successo!")

In [ ]:
# ============================================
# 6️⃣ PROCESSA TUTTI I PDF
# ============================================

print("\n" + "="*60)
print("🚀 INIZIO ELABORAZIONE")
print("="*60)

results = []

for i, pdf_path in enumerate(pdf_files, 1):
    print(f"\n[{i}/{len(pdf_files)}] ", end="")
    result = process_pdf(pdf_path, output_folder)
    results.append(result)

print("\n" + "="*60)
print("✅ ELABORAZIONE COMPLETATA")
print("="*60)

# Statistiche
success_count = sum(1 for r in results if r['status'] == 'success')
error_count = len(results) - success_count

print(f"\n📊 STATISTICHE:")
print(f"   ✅ Successo: {success_count}/{len(results)}")
print(f"   ❌ Errori: {error_count}/{len(results)}")

if error_count > 0:
    print(f"\n⚠️  File con errori:")
    for r in results:
        if r['status'] == 'error':
            print(f"   - {r['original']}: {r['error']}")

In [ ]:
# ============================================
# 7️⃣ SCARICA I FILE RINOMINATI
# ============================================

print("\n📦 Creazione archivio ZIP...")

# Crea ZIP con i file rinominati
zip_filename = f"documenti_rinominati_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for filename in os.listdir(output_folder):
        if filename.endswith('.pdf'):
            filepath = os.path.join(output_folder, filename)
            zipf.write(filepath, filename)

print(f"✅ Archivio creato: {zip_filename}")
print("\n⬇️ Download in corso...\n")

files.download(zip_filename)

print("\n🎉 PROCESSO COMPLETATO!")
print("\nI tuoi file rinominati sono stati scaricati.")
print("Puoi eseguire nuovamente il notebook per processare altri PDF.")

In [ ]:
# ============================================
# 8️⃣ VISUALIZZA REPORT DETTAGLIATO (Opzionale)
# ============================================

import pandas as pd

# Crea DataFrame con i risultati
df_results = pd.DataFrame(results)

print("\n📋 REPORT DETTAGLIATO\n")
print(df_results.to_string(index=False))

# Salva report CSV
report_filename = f"report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_results.to_csv(report_filename, index=False)

print(f"\n💾 Report salvato: {report_filename}")
files.download(report_filename)